# Visualise your datasets with _Matplotlib_

This lab we will focus on using data visualisation techniques to analyse and understand further your loaded datasets. Refer to these (matplotlib tutorials)[https://matplotlib.org/stable/tutorials/index.html] and the Matplotlib documentation for more help.

We will be working with a calcium dataset,#add info about dataset

In [ ]:
!pip install matplotlib

In the previous worksheets we learned how to load a dataset. Time to explore an actual 2p microscope output. 

In [ ]:
#import and load the right library
import numpy as np
import matplotlib as plt

In [ ]:
# calcium dataset -> load
data = np.load("C:/Users/aleja/Downloads/Figures4_5_analysis/python code submitted/processed_data/S1_all_neurons.npy", allow_pickle=True)



# load in and inspect data, some preprocessing, statistics/

In [ ]:
#inspect the structure of the loaded dataset 
type(data)

In [ ]:
# visualise the data
# Plot calcium trace for the first neuron
plt.plot(data[0])
plt.title("Neuron 1 Calcium Trace")
plt.xlabel("Time (frames)")
plt.ylabel("Fluorescence")
plt.show()

In [ ]:
#understand patterns in he data by doing some simple statistics 
mean_trace = np.mean(data, axis=1)   # Mean for each neuron
std_trace = np.std(data, axis=1) # standard deviation for each neuron

print("Mean fluorescence across time (first 5 neurons):", mean_trace[:5])
print("Standard deviation (first 5 neurons):", std_trace[:5])

In [ ]:
# can do some preprocesing by doing z-score normalisation
zscored = (data - np.mean(data, axis=1, keepdims=True)) / np.std(data, axis=1, keepdims=True)

#and Baseline subtraction (ΔF/F)
baseline = np.percentile(data, 20, axis=1, keepdims=True)
dff = (data - baseline) / baseline
